# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

In [17]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import scipy.stats as sps
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set_style('darkgrid')
%matplotlib inline

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
print(df.info())
print(len(df['id'].unique()))
print(df['group'].unique())
print(df['action'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
6328
['experiment' 'control']
['view' 'click']


In [4]:
control_ids = df.loc[df['group'] == 'control']['id'].unique()
experi_ids = df.loc[df['group'] == 'experiment']['id'].unique()
group_overlap = np.intersect1d(control_ids, experi_ids)

print(f'Num of subjects in control group: {len(control_ids)}, Num of subjects in experiment group: {len(experi_ids)}')

print(f'The overlap between the two groups is {len(group_overlap)}')

# there are no nulls in the data set
# there are only two groups (control/experiment) and two actions (view/click)
# No intersection/overlap between control and experiment ids
# The 2 groups are of different sizes....have to do a Welch's T test.

Num of subjects in control group: 3332, Num of subjects in experiment group: 2996
The overlap between the two groups is 0


In [6]:
fig = px.bar(data_frame=df, x='action', color='group')
fig.show()

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [7]:
#Your code here
df_action = pd.get_dummies(df.drop(columns=['id', 'timestamp']), columns=['action'])
df_action.head()

,group,action_click,action_view
0,experiment,0,1
1,experiment,0,1
2,experiment,0,1
3,control,0,1
4,experiment,0,1


In [8]:
df_action_groupby = df_action.groupby(by='group').agg(sum)
df_action_groupby

,action_click,action_view
group,,
control,932.0,3332.0
experiment,928.0,2996.0


In [9]:
mean_clicks_control = 932/(932+3332)
mean_clicks_control

0.21857410881801126

In [10]:
expected_experimental_clicks = mean_clicks_control*(928+2996)
expected_experimental_clicks

857.6848030018762

In [11]:
# actual number is 928 
diff = 928 - expected_experimental_clicks
diff

70.31519699812384

In [12]:
n = 928.0 + 2996.0
p = mean_clicks_control
var = n*p*(1-p)
std = np.sqrt(var)
std

25.888551746650283

In [13]:
std_experimental = diff/std
std_experimental


2.71607302278784

In [19]:
sps.norm.sf(2.71)
#pvalue for the z-score above

0.0033641604066691933

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [ ]:
#Your code here

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.